In [1]:
import requests
from bs4 import BeautifulSoup

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url)

soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [2]:
links

['/en/squads/b8fd03ef/2023-2024/Manchester-City-Stats',
 '/en/squads/18bb7c10/2023-2024/Arsenal-Stats',
 '/en/squads/822bd0ba/2023-2024/Liverpool-Stats',
 '/en/squads/8602292d/2023-2024/Aston-Villa-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/4ba7cbea/Bournemouth-Stats',
 '/en/squads/fd962109/Fulham-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/d3fd31cc/Everton-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/e4a775cb/Nottingham-Forest-Stats',
 '/en/squads/e297cd13/Luton-Town-Stats',
 '/en/squads/943e8050/Burnley-Stats',
 '/en/squads/1df6b87e/Sheffield-United-Stats']

In [3]:
team_urls = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])

In [4]:
import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.head()

/var/folders/bh/54y95t956h793hywb9szy8gh0000gn/T/ipykernel_28527/674890365.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/bh/54y95t956h793hywb9szy8gh0000gn/T/ipykernel_28527/674890365.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2023-08-06  16:00  Community Shield  FA Community Shield  Sun   
1          2023-08-11  20:00    Premier League          Matchweek 1  Fri   
2          2023-08-16  22:00         Super Cup       UEFA Super Cup  Wed   
3          2023-08-19  20:00    Premier League          Matchweek 2  Sat   
4          2023-08-27  14:00    Premier League          Matchweek 3  Sun   

                                                ... Standard                \
     Venue Result     GF     GA       Opponent  ...     Dist   FK PK PKatt   
0  Neutral      D  1 (1)  1 (4)        Arsenal  ...      NaN  NaN  0     0   
1     Away      W      3      0        Burnley  ...     13.9  0.0  0     0   
2     Home      D  1 (5)  1 (4)     es Sevilla  ...      NaN  NaN  0     0   
3     Home      W      1      0  Newcastle Utd  ...     17.9  0.0  0     0   
4     Away      W      2      1  Sheffield Utd  ...     17.3  2.0  0     1   

  Expected                           Unnamed: 25_level_0  
        xG npxG npxG/Sh G-xG np:G-xG        Match Report  
0      NaN  NaN     NaN  NaN     NaN        Match Report  
1      1.9  1.9    0.12  1.1     1.1        Match Report  
2      NaN  NaN     NaN  NaN     NaN        Match Report  
3      1.0  1.0    0.07  0.0     0.0        Match Report  
4      3.5  2.8    0.10 -1.5    -0.8        Match Report  

[5 rows x 26 columns]

In [5]:
shooting.columns = shooting.columns.droplevel()
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,4-2-3-1,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...,8,4,NaN,NaN,0,0
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,4-2-3-1,Craig Pawson,Match Report,NaN,17,8,13.9,0.0,0,0
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,4-2-3-1,François Letexier,Match Report,NaN,23,7,NaN,NaN,0,0
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,4-2-3-1,Robert Jones,Match Report,NaN,14,4,17.9,0.0,0,0
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,4-2-3-1,Jarred Gillett,Match Report,NaN,29,9,17.3,2.0,0,1


In [6]:
years = list(range(2022, 2020, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
import time


for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)


match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df

/var/folders/bh/54y95t956h793hywb9szy8gh0000gn/T/ipykernel_28527/2992476937.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/bh/54y95t956h793hywb9szy8gh0000gn/T/ipykernel_28527/2992476937.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/bh/54y95t956h793hywb9szy8gh0000gn/T/ipykernel_28527/2992476937.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/bh/54y95t956h793hywb9szy8gh0000gn/T/ipyke

ImportError: Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.

In [21]:
match_df.to_csv("matches.csv")

NameError: name 'match_df' is not defined